<a href="https://colab.research.google.com/github/dtoralg/INESDI_Data-Science_ML_IA/blob/main/%5B07%5D%20-%20ML%20en%20la%20Empresa/ML_en_la_empresa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎯 Caso Práctico: Industrializar un modelo


---

## 📋 Información del Módulo

**Asignatura:** Data Analytics: Data Science, Machine Learning e Inteligencia Artificial  
**Máster:** FP en Business Analytics e Inteligencia Artificial  
**Profesores:** Álvaro López Barberá
**Ejemplo Práctico:**  ML en la empresa

---

## 🎓 Objetivo

Desarrollar un modelo de lenguaje natural sobre Machine Learning, que nos conteste a nuestras preguntas.

---

## 📦 PASO 1: Instalar dependencias y Configuración Inicial

In [1]:
!pip install nltk scikit-learn fastapi uvicorn pydantic joblib requests numpy nest-asyncio

In [2]:
import nltk
nltk.download('all')  # Descarga todo (tarda ~5 min)

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_r

True

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import joblib
import numpy as np

In [4]:
# Descargar recursos de NLTK
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

print("✅ Recursos descargados")

✅ Recursos descargados


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


---

## 📊 PASO 2: Crear Base de Conocimiento

**Salida esperada:**
```
CONSTRUYENDO BASE DE CONOCIMIENTO
✓ Total de preguntas en la base: 45
✓ Vectorizando preguntas con TF-IDF...
✓ Vectorización completada
💾 Guardando chatbot en chatbot_data.pkl...
✅ Chatbot guardado exitosamente

In [5]:
## 📊 PASO 2: Crear Base de Conocimiento

### Archivo: `01_crear_base_conocimiento.py`


"""
PASO 1: CREAR BASE DE CONOCIMIENTO DEL CHATBOT
Creamos una base de preguntas-respuestas sobre Machine Learning
y la preparamos para calcular similitudes
"""

import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import joblib
import numpy as np

# ============================================================================
# CONFIGURACIÓN INICIAL: Descargar todos los recursos de NLTK
# ============================================================================
print("\n" + "=" * 60)
print("CONFIGURACIÓN INICIAL")
print("=" * 60)

print("\nDescargando recursos de NLTK (puede tardar unos segundos)...")
recursos_nltk = ['punkt', 'punkt_tab', 'stopwords', 'wordnet', 'omw-1.4']

for recurso in recursos_nltk:
    try:
        nltk.download(recurso, quiet=True)
        print(f"  ✓ {recurso} descargado")
    except Exception as e:
        print(f"  ⚠ Error descargando {recurso}: {e}")

print("\n✅ Recursos de NLTK listos\n")

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

class ChatbotMLFAQ:
    """
    Chatbot simple basado en similitud de texto para FAQs de Machine Learning
    """

    def __init__(self):
        # Base de conocimiento: preguntas y respuestas sobre ML
        self.qa_pairs = [
            {
                "preguntas": [
                    "¿Qué es machine learning?",
                    "¿Qué es el aprendizaje automático?",
                    "Define machine learning",
                    "Explica qué es ML"
                ],
                "respuesta": "Machine Learning es una rama de la Inteligencia Artificial que permite a las máquinas aprender de los datos sin ser programadas explícitamente. Los algoritmos identifican patrones y toman decisiones basándose en ejemplos anteriores."
            },
            {
                "preguntas": [
                    "¿Qué es overfitting?",
                    "¿Qué es el sobreajuste?",
                    "Explica overfitting",
                    "Qué significa sobreajustar"
                ],
                "respuesta": "Overfitting ocurre cuando un modelo aprende demasiado bien los datos de entrenamiento, incluyendo el ruido. Esto hace que funcione muy bien con los datos de entrenamiento pero mal con datos nuevos. Se soluciona con validación cruzada, regularización o más datos."
            },
            {
                "preguntas": [
                    "¿Qué es una red neuronal?",
                    "Define red neuronal",
                    "Explica las redes neuronales",
                    "Qué son las neural networks"
                ],
                "respuesta": "Una red neuronal es un modelo de aprendizaje profundo inspirado en el cerebro humano. Está compuesta por capas de neuronas artificiales conectadas que procesan información. Se usa para tareas complejas como reconocimiento de imágenes o procesamiento de lenguaje natural."
            },
            {
                "preguntas": [
                    "¿Qué diferencia hay entre clasificación y regresión?",
                    "Clasificación vs regresión",
                    "Diferencia entre clasificación y regresión",
                    "Cuándo usar clasificación o regresión"
                ],
                "respuesta": "La clasificación predice categorías o clases (ej: spam/no spam, gato/perro), mientras que la regresión predice valores numéricos continuos (ej: precio de una casa, temperatura). Clasificación da etiquetas, regresión da números."
            },
            {
                "preguntas": [
                    "¿Qué es el accuracy?",
                    "Define accuracy",
                    "Qué mide el accuracy",
                    "Explica la métrica accuracy"
                ],
                "respuesta": "Accuracy (precisión) es el porcentaje de predicciones correctas sobre el total de predicciones. Se calcula como: (predicciones correctas / total de predicciones) × 100. Es útil cuando las clases están balanceadas, pero puede engañar con datos desbalanceados."
            },
            {
                "preguntas": [
                    "¿Qué es gradient descent?",
                    "Explica gradient descent",
                    "Qué es el descenso de gradiente",
                    "Cómo funciona gradient descent"
                ],
                "respuesta": "Gradient Descent es un algoritmo de optimización que ajusta los parámetros de un modelo para minimizar el error. Funciona calculando el gradiente (derivada) de la función de pérdida y moviéndose en la dirección opuesta para encontrar el mínimo."
            },
            {
                "preguntas": [
                    "¿Qué es cross-validation?",
                    "Explica la validación cruzada",
                    "Para qué sirve cross-validation",
                    "Qué es k-fold"
                ],
                "respuesta": "Cross-validation es una técnica para evaluar modelos dividing los datos en k partes (folds). Se entrena k veces usando k-1 partes para entrenar y 1 para validar, rotando. Esto da una estimación más robusta del rendimiento real del modelo."
            },
            {
                "preguntas": [
                    "¿Qué es un Random Forest?",
                    "Explica Random Forest",
                    "Cómo funciona Random Forest",
                    "Qué es bosque aleatorio"
                ],
                "respuesta": "Random Forest es un modelo ensemble que combina múltiples árboles de decisión. Cada árbol se entrena con una muestra aleatoria de datos y features. La predicción final es el promedio (regresión) o voto mayoritario (clasificación) de todos los árboles."
            },
            {
                "preguntas": [
                    "¿Qué es feature engineering?",
                    "Explica feature engineering",
                    "Para qué sirve feature engineering",
                    "Qué es ingeniería de características"
                ],
                "respuesta": "Feature Engineering es el proceso de crear nuevas características (features) a partir de los datos existentes para mejorar el rendimiento del modelo. Incluye transformaciones, combinaciones, extracciones y selección de las variables más relevantes."
            },
            {
                "preguntas": [
                    "¿Qué es supervised learning?",
                    "Explica aprendizaje supervisado",
                    "Qué es supervised learning",
                    "Diferencia entre supervisado y no supervisado"
                ],
                "respuesta": "Supervised Learning es cuando entrenamos un modelo con datos etiquetados (sabemos la respuesta correcta). El modelo aprende la relación entre inputs y outputs. Ejemplos: clasificación de emails, predicción de precios. Se diferencia del no supervisado que trabaja sin etiquetas."
            },
            {
                "preguntas": [
                    "¿Qué es un hiperparámetro?",
                    "Diferencia entre parámetro e hiperparámetro",
                    "Explica hiperparámetros",
                    "Qué son los hyperparameters"
                ],
                "respuesta": "Los hiperparámetros son configuraciones del modelo que definimos ANTES del entrenamiento (ej: learning rate, número de capas). Los parámetros son los valores que el modelo APRENDE durante el entrenamiento (ej: pesos de una red neuronal)."
            },
            {
                "preguntas": [
                    "Hola",
                    "Buenos días",
                    "Buenas tardes",
                    "Hey"
                ],
                "respuesta": "¡Hola! Soy un chatbot especializado en Machine Learning. Puedo responder preguntas sobre conceptos de ML, algoritmos, métricas y más. ¿En qué puedo ayudarte?"
            },
            {
                "preguntas": [
                    "Gracias",
                    "Muchas gracias",
                    "Perfecto gracias",
                    "Ok gracias"
                ],
                "respuesta": "¡De nada! Si tienes más preguntas sobre Machine Learning, estaré encantado de ayudarte. 😊"
            },
            {
                "preguntas": [
                    "Adiós",
                    "Hasta luego",
                    "Chao",
                    "Nos vemos"
                ],
                "respuesta": "¡Hasta pronto! Que tengas un buen día aprendiendo Machine Learning. 🚀"
            }
        ]

        # Preparar lematizador y stopwords
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('spanish'))

        # Vectorizador TF-IDF
        self.vectorizer = None
        self.question_vectors = None
        self.all_questions = []
        self.question_to_answer = {}

    def preprocess_text(self, text):
        """
        Preprocesa el texto: tokeniza, elimina stopwords y lematiza
        """
        try:
            # Convertir a minúsculas
            text = text.lower()

            # Tokenizar
            tokens = word_tokenize(text, language='spanish')

            # Eliminar stopwords y lematizar
            processed_tokens = [
                self.lemmatizer.lemmatize(token)
                for token in tokens
                if token.isalnum() and token not in self.stop_words
            ]

            return ' '.join(processed_tokens)

        except Exception as e:
            # Si falla la tokenización en español, intentar con inglés
            print(f"⚠ Advertencia en preprocesamiento: {e}")
            try:
                tokens = word_tokenize(text)
                processed_tokens = [
                    self.lemmatizer.lemmatize(token)
                    for token in tokens
                    if token.isalnum()
                ]
                return ' '.join(processed_tokens)
            except:
                # Último recurso: dividir por espacios
                return ' '.join(text.lower().split())

    def build_knowledge_base(self):
        """
        Construye la base de conocimiento y vectoriza las preguntas
        """
        print("\n" + "=" * 60)
        print("CONSTRUYENDO BASE DE CONOCIMIENTO")
        print("=" * 60)

        # Crear lista de todas las preguntas y mapeo a respuestas
        for qa_pair in self.qa_pairs:
            respuesta = qa_pair["respuesta"]
            for pregunta in qa_pair["preguntas"]:
                pregunta_procesada = self.preprocess_text(pregunta)
                self.all_questions.append(pregunta_procesada)
                self.question_to_answer[pregunta_procesada] = respuesta

        print(f"\n✓ Total de preguntas en la base: {len(self.all_questions)}")
        print(f"✓ Total de respuestas únicas: {len(self.qa_pairs)}")

        # Vectorizar preguntas usando TF-IDF
        print("\n Vectorizando preguntas con TF-IDF...")
        self.vectorizer = TfidfVectorizer()
        self.question_vectors = self.vectorizer.fit_transform(self.all_questions)

        print("✓ Vectorización completada")

    def get_response(self, user_question, threshold=0.3):
        """
        Obtiene la respuesta más similar a la pregunta del usuario

        Args:
            user_question: Pregunta del usuario
            threshold: Umbral mínimo de similitud (0-1)

        Returns:
            Respuesta del chatbot y score de confianza
        """
        # Preprocesar pregunta del usuario
        processed_question = self.preprocess_text(user_question)

        # Vectorizar pregunta del usuario
        user_vector = self.vectorizer.transform([processed_question])

        # Calcular similitud con todas las preguntas
        similarities = cosine_similarity(user_vector, self.question_vectors)[0]

        # Encontrar la pregunta más similar
        best_match_idx = np.argmax(similarities)
        best_similarity = similarities[best_match_idx]

        # Si la similitud es muy baja, no sabemos la respuesta
        if best_similarity < threshold:
            return {
                "answer": "Lo siento, no tengo información sobre eso. Intenta preguntarme sobre conceptos de Machine Learning como overfitting, redes neuronales, gradient descent, etc.",
                "confidence": float(best_similarity),
                "matched_question": None
            }

        # Obtener la respuesta
        matched_question = self.all_questions[best_match_idx]
        answer = self.question_to_answer[matched_question]

        return {
            "answer": answer,
            "confidence": float(best_similarity),
            "matched_question": matched_question
        }

    def save(self, filename="chatbot_data.pkl"):
        """
        Guarda el chatbot completo
        """
        print(f"\n💾 Guardando chatbot en {filename}...")
        joblib.dump({
            'vectorizer': self.vectorizer,
            'question_vectors': self.question_vectors,
            'all_questions': self.all_questions,
            'question_to_answer': self.question_to_answer,
            'lemmatizer': self.lemmatizer,
            'stop_words': self.stop_words
        }, filename)
        print(f"✅ Chatbot guardado exitosamente")

def main():
    """
    Función principal para crear y guardar el chatbot
    """
    print("\n" + "█" * 60)
    print(" CREACIÓN DE CHATBOT ML - BASE DE CONOCIMIENTO")
    print("█" * 60)

    # Crear chatbot
    chatbot = ChatbotMLFAQ()

    # Construir base de conocimiento
    chatbot.build_knowledge_base()

    # Guardar chatbot
    chatbot.save()

    # Prueba rápida
    print("\n" + "=" * 60)
    print("PRUEBA RÁPIDA DEL CHATBOT")
    print("=" * 60)

    test_questions = [
        "¿Qué es machine learning?",
        "Explícame overfitting",
        "¿Cómo funciona gradient descent?"
    ]

    for question in test_questions:
        print(f"\n❓ Pregunta: {question}")
        response = chatbot.get_response(question)
        print(f"🤖 Respuesta: {response['answer'][:100]}...")
        print(f"📊 Confianza: {response['confidence']:.2%}")

    print("\n" + "█" * 60)
    print(" ✅ CHATBOT CREADO Y GUARDADO EXITOSAMENTE")
    print("█" * 60)
    print("\nAhora ejecuta: python 02_api_chatbot.py")

if __name__ == "__main__":
    main()



CONFIGURACIÓN INICIAL

Descargando recursos de NLTK (puede tardar unos segundos)...
  ✓ punkt descargado
  ✓ punkt_tab descargado
  ✓ stopwords descargado
  ✓ wordnet descargado
  ✓ omw-1.4 descargado

✅ Recursos de NLTK listos


████████████████████████████████████████████████████████████
 CREACIÓN DE CHATBOT ML - BASE DE CONOCIMIENTO
████████████████████████████████████████████████████████████

CONSTRUYENDO BASE DE CONOCIMIENTO

✓ Total de preguntas en la base: 56
✓ Total de respuestas únicas: 14

 Vectorizando preguntas con TF-IDF...
✓ Vectorización completada

💾 Guardando chatbot en chatbot_data.pkl...
✅ Chatbot guardado exitosamente

PRUEBA RÁPIDA DEL CHATBOT

❓ Pregunta: ¿Qué es machine learning?
🤖 Respuesta: Machine Learning es una rama de la Inteligencia Artificial que permite a las máquinas aprender de lo...
📊 Confianza: 100.00%

❓ Pregunta: Explícame overfitting
🤖 Respuesta: Overfitting ocurre cuando un modelo aprende demasiado bien los datos de entrenamiento, incluyendo el.

---

## 🌐 PASO 3: API REST del Chatbot

**Salida esperada:**
```
INICIANDO API DEL CHATBOT ML
📂 Cargando chatbot desde chatbot_data.pkl...
✅ Chatbot cargado exitosamente
   - Preguntas en base: 45

Iniciando servidor en: http://127.0.0.1:8000
Documentación: http://127.0.0.1:8000/docs

In [6]:
## 🌐 PASO 3: API REST del Chatbot

### Archivo: `02_api_chatbot.py`

"""
PASO 2: API REST PARA EL CHATBOT
Expone el chatbot a través de una API REST con FastAPI
"""

from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel, Field
import joblib
from typing import Optional
import uvicorn
import os

# ============================================================================
# CONFIGURACIÓN DE LA API
# ============================================================================

app = FastAPI(
    title="ML FAQ Chatbot API",
    description="API de chatbot para preguntas sobre Machine Learning",
    version="1.0.0"
)

# Configurar CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Variable global para el chatbot
chatbot_data = None

# ============================================================================
# MODELOS DE DATOS
# ============================================================================

class ChatRequest(BaseModel):
    """
    Modelo de entrada: pregunta del usuario
    """
    question: str = Field(
        ...,
        description="Pregunta del usuario sobre Machine Learning",
        min_length=1,
        max_length=500
    )
    threshold: Optional[float] = Field(
        0.3,
        description="Umbral mínimo de confianza (0-1)",
        ge=0.0,
        le=1.0
    )

    class Config:
        json_schema_extra = {
            "example": {
                "question": "¿Qué es overfitting?",
                "threshold": 0.3
            }
        }

class ChatResponse(BaseModel):
    """
    Modelo de respuesta del chatbot
    """
    question: str
    answer: str
    confidence: float
    matched_question: Optional[str]
    status: str

# ============================================================================
# CLASE CHATBOT (versión para API)
# ============================================================================

class ChatbotAPI:
    """
    Versión del chatbot para usar en la API
    """

    def __init__(self, data_file="chatbot_data.pkl"):
        """
        Carga el chatbot desde el archivo guardado
        """
        if not os.path.exists(data_file):
            raise FileNotFoundError(
                f"No se encontró {data_file}. "
                f"Ejecuta primero: python 01_crear_base_conocimiento.py"
            )

        print(f"📂 Cargando chatbot desde {data_file}...")
        data = joblib.load(data_file)

        self.vectorizer = data['vectorizer']
        self.question_vectors = data['question_vectors']
        self.all_questions = data['all_questions']
        self.question_to_answer = data['question_to_answer']
        self.lemmatizer = data['lemmatizer']
        self.stop_words = data['stop_words']

        print("✅ Chatbot cargado exitosamente")
        print(f"   - Preguntas en base: {len(self.all_questions)}")
        print(f"   - Respuestas únicas: {len(set(self.question_to_answer.values()))}")

    def preprocess_text(self, text):
        """
        Preprocesa el texto
        """
        from nltk.tokenize import word_tokenize

        text = text.lower()
        tokens = word_tokenize(text, language='spanish')
        processed_tokens = [
            self.lemmatizer.lemmatize(token)
            for token in tokens
            if token.isalnum() and token not in self.stop_words
        ]
        return ' '.join(processed_tokens)

    def get_response(self, user_question, threshold=0.3):
        """
        Obtiene respuesta para la pregunta del usuario
        """
        from sklearn.metrics.pairwise import cosine_similarity
        import numpy as np

        # Preprocesar pregunta
        processed_question = self.preprocess_text(user_question)

        # Vectorizar
        user_vector = self.vectorizer.transform([processed_question])

        # Calcular similitud
        similarities = cosine_similarity(user_vector, self.question_vectors)[0]

        # Mejor coincidencia
        best_match_idx = np.argmax(similarities)
        best_similarity = similarities[best_match_idx]

        # Verificar umbral
        if best_similarity < threshold:
            return {
                "answer": "Lo siento, no tengo información sobre eso. Intenta preguntarme sobre conceptos de Machine Learning como overfitting, redes neuronales, gradient descent, Random Forest, etc.",
                "confidence": float(best_similarity),
                "matched_question": None,
                "status": "low_confidence"
            }

        # Obtener respuesta
        matched_question = self.all_questions[best_match_idx]
        answer = self.question_to_answer[matched_question]

        return {
            "answer": answer,
            "confidence": float(best_similarity),
            "matched_question": matched_question,
            "status": "success"
        }

# ============================================================================
# EVENTOS DE LA API
# ============================================================================

@app.on_event("startup")
async def startup_event():
    """
    Carga el chatbot al iniciar la API
    """
    global chatbot_data

    print("\n" + "=" * 60)
    print("INICIANDO API DEL CHATBOT ML")
    print("=" * 60 + "\n")

    try:
        chatbot_data = ChatbotAPI()
    except FileNotFoundError as e:
        print(f"\n❌ ERROR: {e}\n")
        raise

# ============================================================================
# ENDPOINTS
# ============================================================================

@app.get("/")
async def root():
    """
    Endpoint raíz - información de la API
    """
    return {
        "message": "ML FAQ Chatbot API",
        "version": "1.0.0",
        "description": "Chatbot especializado en preguntas sobre Machine Learning",
        "endpoints": {
            "chat": "/chat (POST)",
            "health": "/health (GET)",
            "stats": "/stats (GET)",
            "examples": "/examples (GET)",
            "docs": "/docs (GET)"
        }
    }

@app.get("/health")
async def health():
    """
    Verificación de salud de la API
    """
    if chatbot_data is None:
        raise HTTPException(status_code=503, detail="Chatbot no cargado")

    return {
        "status": "healthy",
        "chatbot_loaded": True,
        "questions_in_base": len(chatbot_data.all_questions)
    }

@app.get("/stats")
async def stats():
    """
    Estadísticas del chatbot
    """
    if chatbot_data is None:
        raise HTTPException(status_code=503, detail="Chatbot no cargado")

    return {
        "total_questions": len(chatbot_data.all_questions),
        "unique_answers": len(set(chatbot_data.question_to_answer.values())),
        "topics": [
            "Machine Learning básico",
            "Overfitting y validación",
            "Algoritmos (Random Forest, Gradient Descent)",
            "Métricas (Accuracy)",
            "Conceptos (Feature Engineering, Hiperparámetros)"
        ]
    }

@app.get("/examples")
async def examples():
    """
    Ejemplos de preguntas que el chatbot puede responder
    """
    return {
        "examples": [
            "¿Qué es machine learning?",
            "Explícame qué es overfitting",
            "¿Qué diferencia hay entre clasificación y regresión?",
            "¿Qué es gradient descent?",
            "¿Para qué sirve cross-validation?",
            "¿Qué es Random Forest?",
            "Explica feature engineering",
            "¿Qué es un hiperparámetro?"
        ]
    }

@app.post("/chat", response_model=ChatResponse)
async def chat(request: ChatRequest):
    """
    Endpoint principal del chatbot

    Recibe una pregunta y devuelve la respuesta del chatbot
    """
    if chatbot_data is None:
        raise HTTPException(status_code=503, detail="Chatbot no cargado")

    try:
        # Obtener respuesta del chatbot
        response = chatbot_data.get_response(
            request.question,
            threshold=request.threshold
        )

        return ChatResponse(
            question=request.question,
            answer=response["answer"],
            confidence=response["confidence"],
            matched_question=response["matched_question"],
            status=response["status"]
        )

    except Exception as e:
        raise HTTPException(
            status_code=500,
            detail=f"Error al procesar la pregunta: {str(e)}"
        )

# ============================================================================
# EJECUTAR SERVIDOR
# ============================================================================

if __name__ == "__main__":
    print("\n" + "=" * 60)
    print("SERVIDOR API - ML FAQ Chatbot")
    print("=" * 60)
    print("\nIniciando servidor en: http://127.0.0.1:8000")
    print("Documentación: http://127.0.0.1:8000/docs")
    print("\nPresiona CTRL+C para detener\n")

    # Configuración para evitar conflictos con event loops existentes
    import sys

    # Detectar si estamos en Jupyter/Colab
    try:
        get_ipython()
        IN_NOTEBOOK = True
    except NameError:
        IN_NOTEBOOK = False

    if IN_NOTEBOOK:
        # En Jupyter/Colab, usar nest_asyncio
        print("⚠️  Detectado entorno Jupyter/Colab")
        print("   Ejecutando servidor en modo compatible...\n")

        try:
            import nest_asyncio
            nest_asyncio.apply()
        except ImportError:
            print("❌ Instalando nest_asyncio...")
            import subprocess
            subprocess.check_call([sys.executable, "-m", "pip", "install", "nest_asyncio"])
            import nest_asyncio
            nest_asyncio.apply()

        # Iniciar servidor en segundo plano
        import threading

        def run_server():
            uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

        server_thread = threading.Thread(target=run_server, daemon=True)
        server_thread.start()

        print("✅ Servidor iniciado en segundo plano")
        print("   Accede a: http://127.0.0.1:8000/docs")
        print("   El servidor se detendrá cuando cierres el notebook\n")
    else:
        # En terminal normal
        uvicorn.run(
            app,
            host="0.0.0.0",
            port=8000,
            log_level="info"
        )

/tmp/ipython-input-4206241009.py:171: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")



SERVIDOR API - ML FAQ Chatbot

Iniciando servidor en: http://127.0.0.1:8000
Documentación: http://127.0.0.1:8000/docs

Presiona CTRL+C para detener

⚠️  Detectado entorno Jupyter/Colab
   Ejecutando servidor en modo compatible...

✅ Servidor iniciado en segundo plano
   Accede a: http://127.0.0.1:8000/docs
   El servidor se detendrá cuando cierres el notebook



---

## 🧪 PASO 4: Cliente de Prueba

In [7]:
## 🧪 PASO 4: Cliente de Prueba

### Archivo: `03_test_chatbot.py`
"""
PASO 3: CLIENTE PARA PROBAR EL CHATBOT
Script para probar la API del chatbot
"""

import requests
import json

BASE_URL = "http://127.0.0.1:8000"

def print_response(title, response):
    """
    Imprime la respuesta de manera bonita
    """
    print("\n" + "=" * 70)
    print(f"  {title}")
    print("=" * 70)
    print(f"Status: {response.status_code}")

    if response.status_code == 200:
        data = response.json()
        print(json.dumps(data, indent=2, ensure_ascii=False))
    else:
        print(f"Error: {response.text}")

def test_health():
    """
    Test del health check
    """
    response = requests.get(f"{BASE_URL}/health")
    print_response("TEST 1: Health Check", response)

def test_stats():
    """
    Test de estadísticas
    """
    response = requests.get(f"{BASE_URL}/stats")
    print_response("TEST 2: Estadísticas del Chatbot", response)

def test_examples():
    """
    Test de ejemplos
    """
    response = requests.get(f"{BASE_URL}/examples")
    print_response("TEST 3: Preguntas de Ejemplo", response)

def test_chat_questions():
    """
    Test de preguntas al chatbot
    """
    questions = [
        "¿Qué es machine learning?",
        "Explícame el overfitting por favor",
        "¿Cuál es la diferencia entre clasificación y regresión?",
        "¿Qué es el gradient descent?",
        "Hola chatbot",
        "Gracias por la ayuda",
        "¿Qué es la física cuántica?"  # Esta NO debería saber
    ]

    for i, question in enumerate(questions, 1):
        print(f"\n{'='*70}")
        print(f"  TEST 4.{i}: Pregunta al Chatbot")
        print(f"{'='*70}")
        print(f"❓ Pregunta: {question}")

        response = requests.post(
            f"{BASE_URL}/chat",
            json={"question": question}
        )

        if response.status_code == 200:
            data = response.json()
            print(f"\n🤖 Respuesta:")
            print(f"   {data['answer']}")
            print(f"\n📊 Confianza: {data['confidence']:.2%}")
            print(f"📌 Estado: {data['status']}")
            if data['matched_question']:
                print(f"🔍 Pregunta similar: {data['matched_question']}")
        else:
            print(f"❌ Error: {response.text}")

def run_all_tests():
    """
    Ejecuta todos los tests
    """
    print("\n" + "█" * 70)
    print("  SUITE DE TESTS - ML FAQ CHATBOT API")
    print("█" * 70)

    try:
        test_health()
        test_stats()
        test_examples()
        test_chat_questions()

        print("\n" + "█" * 70)
        print("  ✅ TODOS LOS TESTS COMPLETADOS")
        print("█" * 70 + "\n")

    except requests.exceptions.ConnectionError:
        print("\n❌ ERROR: No se puede conectar a la API")
        print("Asegúrate de que el servidor esté corriendo:")
        print("   python 02_api_chatbot.py\n")
    except Exception as e:
        print(f"\n❌ ERROR: {e}\n")

if __name__ == "__main__":
    run_all_tests()

INFO:     Started server process [3110]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)



INICIANDO API DEL CHATBOT ML

📂 Cargando chatbot desde chatbot_data.pkl...
✅ Chatbot cargado exitosamente
   - Preguntas en base: 56
   - Respuestas únicas: 14

██████████████████████████████████████████████████████████████████████
  SUITE DE TESTS - ML FAQ CHATBOT API
██████████████████████████████████████████████████████████████████████
INFO:     127.0.0.1:35750 - "GET /health HTTP/1.1" 200 OK

  TEST 1: Health Check
Status: 200
{
  "status": "healthy",
  "chatbot_loaded": true,
  "questions_in_base": 56
}
INFO:     127.0.0.1:35762 - "GET /stats HTTP/1.1" 200 OK

  TEST 2: Estadísticas del Chatbot
Status: 200
{
  "total_questions": 56,
  "unique_answers": 14,
  "topics": [
    "Machine Learning básico",
    "Overfitting y validación",
    "Algoritmos (Random Forest, Gradient Descent)",
    "Métricas (Accuracy)",
    "Conceptos (Feature Engineering, Hiperparámetros)"
  ]
}
INFO:     127.0.0.1:35768 - "GET /examples HTTP/1.1" 200 OK

  TEST 3: Preguntas de Ejemplo
Status: 200
{
  "exa

---

## 🧪 PASO 5: Lanzar el servidor en local

In [8]:
# ========================================
# EXPONER SERVIDOR EN GOOGLE COLAB
# ========================================

# Usar el proxy de Colab
from google.colab.output import eval_js
print("🔄 Obteniendo URL del servidor...")

# La URL base de Colab
colab_url = eval_js("google.colab.kernel.proxyPort(8000)")

print("\n" + "✅" * 35)
print("\n   🎉 SERVIDOR ACTIVO EN COLAB 🎉")
print("\n" + "✅" * 35)
print(f"\n🌐 URL del servidor:")
print(f"   {colab_url}")
print(f"\n📖 ABRE ESTA URL EN TU NAVEGADOR:")
print(f"   👉 {colab_url}/docs")
print(f"\n📍 Otros endpoints:")
print(f"   • Health:   {colab_url}/health")
print(f"   • Chat:     {colab_url}/chat")
print("\n" + "=" * 70)
print("\n💡 Click en la URL para abrir en nueva pestaña")
print("=" * 70)

🔄 Obteniendo URL del servidor...

✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅

   🎉 SERVIDOR ACTIVO EN COLAB 🎉

✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅

🌐 URL del servidor:
   https://8000-m-s-1b5it1befvjgq-d.us-east1-1.prod.colab.dev

📖 ABRE ESTA URL EN TU NAVEGADOR:
   👉 https://8000-m-s-1b5it1befvjgq-d.us-east1-1.prod.colab.dev/docs

📍 Otros endpoints:
   • Health:   https://8000-m-s-1b5it1befvjgq-d.us-east1-1.prod.colab.dev/health
   • Chat:     https://8000-m-s-1b5it1befvjgq-d.us-east1-1.prod.colab.dev/chat


💡 Click en la URL para abrir en nueva pestaña



## 💡 CÓMO FUNCIONA EL CHATBOT

### 1. Procesamiento de Texto (NLP)

```python
Pregunta original: "¿Qué es el overfitting?"
                    ↓
Preprocesamiento:  - Minúsculas
                   - Tokenización
                   - Eliminar stopwords ("qué", "es", "el")
                   - Lematización
                    ↓
Texto procesado:   "overfitting"
```

### 2. Vectorización TF-IDF

```
TF-IDF convierte texto en números:

"overfitting" → [0.0, 0.0, 0.87, 0.0, 0.0, ...]
                 (vector de 100+ dimensiones)
```

### 3. Similitud Coseno

```python
Pregunta usuario:    [0.0, 0.0, 0.87, ...]
                            ↓
         Calcular similitud con TODAS las preguntas
                            ↓
Pregunta 1: 0.23 ←
Pregunta 2: 0.95 ← ¡MEJOR MATCH!
Pregunta 3: 0.15 ←
                            ↓
            Devolver respuesta de Pregunta 2
```

---


## 🎓 EJERCICIOS INTERESANTES

### Ejercicio 1: Añadir más preguntas (Fácil)
Modifica `01_crear_base_conocimiento.py` para añadir 3 nuevas preguntas sobre Deep Learning.

### Ejercicio 2: Ajustar el threshold (Medio)
Experimenta cambiando el threshold en `/chat`. ¿Qué pasa si lo subes a 0.7? ¿Y si lo bajas a 0.1?

### Ejercicio 3: Endpoint de feedback (Medio)
Crea un nuevo endpoint `/feedback` que permita al usuario indicar si la respuesta fue útil.

### Ejercicio 4: Historial de conversación (Avanzado)
Implementa un sistema que guarde el historial de preguntas en memoria para mantener contexto.

---

## ✅ CHECKLIST DE LA SESIÓN

- [ ] Entender qué es industrialización de modelos
- [ ] Conocer conceptos básicos de NLP (tokenización, TF-IDF)
- [ ] Comprender cómo funciona similitud coseno
- [ ] Crear una base de conocimiento
- [ ] Serializar datos con pickle
- [ ] Crear una API REST con FastAPI
- [ ] Probar la API con diferentes métodos
- [ ] Explorar documentación automática (Swagger)
- [ ] Modificar y extender el chatbot

---

## 📚 RECURSOS ADICIONALES

### Documentación:
- **NLTK**: https://www.nltk.org/
- **Scikit-learn TfidfVectorizer**: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
- **FastAPI**: https://fastapi.tiangolo.com/
- **Cosine Similarity**: https://en.wikipedia.org/wiki/Cosine_similarity

### Tutoriales:
- Introduction to NLP with NLTK
- Building REST APIs with FastAPI
- Text Similarity with TF-IDF

## 🎓 PREGUNTAS FRECUENTES

**P: ¿Por qué usar TF-IDF y no simplemente contar palabras?**
R: TF-IDF da más peso a palabras importantes y menos a palabras comunes. "Overfitting" es más importante que "qué" o "es".

**P: ¿Esto es un modelo de Machine Learning real?**
R: No es un modelo que "aprende", pero usa técnicas de ML (vectorización, similitud). Es perfecto para entender industrialización.

**P: ¿Puedo usarlo para otros idiomas?**
R: Sí, solo cambia `language='spanish'` por `language='english'` en el código y ajusta los stopwords.

**P: ¿Cómo lo escalo para miles de preguntas?**
R: Considera usar Elasticsearch o FAISS para búsquedas vectoriales más eficientes.

**P: ¿Funciona con WhatsApp/Telegram?**
R: Sí, puedes integrar la API con cualquier plataforma usando sus APIs (ej: Twilio para WhatsApp).


## 🎉 MENSAJE FINAL

Este proyecto demuestra que industrializar un modelo no siempre requiere infraestructura compleja. Con FastAPI + pickle + un poco de NLP, puedes crear un servicio útil en menos de 200 líneas de código.

**Lo más importante:** Los estudiantes ven el ciclo completo:
1. Crear conocimiento → 2. Guardarlo → 3. Exponerlo como API → 4. Consumirlo

¡Esto es industrialización en su forma más práctica!